<a href="https://colab.research.google.com/github/vat99/makemore/blob/main/micrograd_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [2]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29


def gradf(a, b, c):
  df_da = -3*(a**2) - 0.5*(a**-0.5)
  df_db = 3*cos(3*b) + 2.5*(b**1.5)
  df_dc = c**-2
  return [df_da, df_db, df_dc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [9]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
# TODO
h = 1e-10
a, b, c = 2, 3, 4
delta_a = (f(a+h, b, c)-f(a, b, c))/h
delta_b = (f(a, b+h, c)-f(a, b, c))/h
delta_c = (f(a, b, c+h)-f(a, b, c))/h
numerical_grad = [delta_a, delta_b, delta_c]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.35355817641448
OK for dim 1: expected 10.25699027111255, yours returns 10.256986371359744
OK for dim 2: expected 0.0625, yours returns 0.06250111539429781


In [14]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
# TODO
h = 1e-3
a, b, c = 2, 3, 4
delta_a = (f(a+h, b, c)-f(a-h, b, c))/(2*h)
delta_b = (f(a, b+h, c)-f(a, b-h, c))/(2*h)
delta_c = (f(a, b, c+h)-f(a, b, c-h))/(2*h)
numerical_grad2 = [delta_a, delta_b, delta_c]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353554401639766
OK for dim 1: expected 10.25699027111255, yours returns 10.256994551617105
OK for dim 2: expected 0.0625, yours returns 0.06250000390650712


## section 2: support for softmax

In [94]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out
  
  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  # ------
  def __radd__(self, other):
    return self + other

  def exp(self):
    out = Value(exp(self.data), (self, ), 'exp')

    def _backward():
      self.grad += exp(self.data) * out.grad
    out._backward = _backward
    
    return out

  def __neg__(self):
    return self * -1

  def my__pow__(self, other):
    assert isinstance(other, (int, float))
    out = Value(self.data ** other, (self, ), f'**{other}')
    def _backward():
      self.grad += other * (self.data ** (other-1)) * out.grad
    out._backward = _backward
    return out

  def __pow__(self, k):
    if isinstance(k, Value):
      raise 'pow only supported for scalars for now'

    out = Value(self.data ** k, (self, ), '**')
    def _backward():
      self.grad += k * self.data**(k-1) * out.grad
    
    out._backward = _backward
    return out

  def __truediv__(self, other):
    return self * other ** -1

  # def __rdiv__(self, other):
  #   return self * (other ** -1)

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    
    return out

  def __rmul__(self, other):
    return self * other

  def log(self):
    l = log(self.data)
    out = Value(l, (self,), 'log')
    def _backward():
      self.grad += out.grad / self.data
    
    out._backward = _backward
    return out

  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [95]:
Value(0.0) / Value(1.0)
Value(0.0) * Value(1.0)
Value(2.0).log()

Value(data=0.6931471805599453)

In [96]:
# lets test this shit
logits = [Value(10.0), Value(3.0)]
probs = softmax(logits)
probs[1].backward()
print(probs)
print(f"[{logits[0].grad}, {logits[1].grad}]")

#### pytorch version
import torch
logitst = torch.Tensor([10, 3]); logitst.requires_grad = True
probs = torch.softmax(logitst, dim=0)
print(probs)
probs[1].backward()
print(logitst.grad.data)

counts: [Value(data=22026.465794806718), Value(data=20.085536923187668)]
denominator: Value(data=22046.551331729905)
[Value(data=0.9990889488055994), Value(data=0.0009110511944006454)]
[-0.0009102211801218266, 0.0009102211801218265]
tensor([9.9909e-01, 9.1105e-04], grad_fn=<SoftmaxBackward0>)
tensor([-0.0009,  0.0009])


In [97]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  print(f'counts: {counts}')
  denominator = sum(counts, 0)
  print(f"denominator: {denominator}")
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
print(probs)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


counts: [Value(data=1.0), Value(data=20.085536923187668), Value(data=0.1353352832366127), Value(data=2.718281828459045)]
denominator: Value(data=23.939154034883327)
[Value(data=0.04177257051535045), Value(data=0.839024507462532), Value(data=0.00565330266221633), Value(data=0.11354961935990122)]
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [98]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
logits = torch.tensor([0.0, 3.0, -2.0, 1.0])
logits.requires_grad = True
targets = torch.tensor([0.0, 0.0, 0.0, 1.0])

counts = torch.exp(logits)
print(counts)
denominator = torch.sum(counts)
print(denominator)
probs = counts / denominator
print(probs)
loss = -1 * torch.log(torch.sum(probs * targets))
print(loss)
loss.backward()
print(logits.grad)

tensor([ 1.0000, 20.0855,  0.1353,  2.7183], grad_fn=<ExpBackward0>)
tensor(23.9392, grad_fn=<SumBackward0>)
tensor([0.0418, 0.8390, 0.0057, 0.1135], grad_fn=<DivBackward0>)
tensor(2.1755, grad_fn=<MulBackward0>)
tensor([ 0.0418,  0.8390,  0.0057, -0.8865])


In [99]:
import torch.nn.functional as F

logits = torch.tensor([0.0, 3.0, -2.0, 1.0])
print(f"logits: {logits}")
softmax_output = F.softmax(logits, dim=0)
print(f"softmax_output {softmax_output}")
print(softmax_output.sum())

logits: tensor([ 0.,  3., -2.,  1.])
softmax_output tensor([0.0418, 0.8390, 0.0057, 0.1135])
tensor(1.0000)
